# Meteo forcing for HNMS gribs

This is the prototype for analysing HNMS files.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
from pyresample import bilinear, geometry, kd_tree
from pyresample import utils
import pandas as pd
import glob
import pyproj
from gribapi import *
import datetime
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import xarray as xr

In [ ]:
hv.extension('bokeh')

In [ ]:
%matplotlib notebook

In [ ]:
folder= '/Users/brey/Downloads/HNMS/'

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-5.5, # lat/lon window
     'maxlon':41.8,
     'minlat':30.2,
     'maxlat':47.5,
     'start_date':'2018-1-1',
     'end_date':'2018-1-1 12'
    }

In [ ]:
dr = pd.date_range(dic['start_date'],dic['end_date'], freq='12H')
dr

In [ ]:
#creating a sequence of folder from which we read the meteo.
PATH='/Users/brey/Downloads/HNMS/01/'
files = glob.glob(PATH+'/E_JRC00*')

In [ ]:
files.sort()

In [ ]:
files

In [ ]:
filenames = files[:10]
minlon = -5.5 #kwargs.get('minlon', None)
maxlon = 41.8 #kwargs.get('maxlon', None)
minlat = 30.2 #kwargs.get('minlat', None)
maxlat = 47.5 #kwargs.get('maxlat', None) 
 
# read grib file and append to xarray
pt=[]
ut=[]
vt=[]
tt=[]

for filename in filenames:

    try: 
        f = open(filename)
    except:
        print 'no file {}'.format(filename)
        sys.exit(1)


      #--------------------------------------------------------------------- 
    sys.stdout.flush()
    sys.stdout.write('\n')
    sys.stdout.write('extracting meteo from {}\n'.format(filename))
    sys.stdout.flush()
      #---------------------------------------------------------------------      

    try:    
        for it in range(3):
        
            gid = grib_new_from_file(f)
    
            name=grib_get(gid, 'shortName')
            mv=grib_get(gid,'missingValue')

            date=grib_get(gid, 'date')
            dataTime=grib_get(gid, 'dataTime')
            stepRange=grib_get(gid, 'stepRange')

            lonfgp=grib_get(gid,'longitudeOfFirstGridPointInDegrees')
            latfgp=grib_get(gid,'latitudeOfFirstGridPointInDegrees')
            lonlgp=grib_get(gid,'longitudeOfLastGridPointInDegrees')
            latlgp=grib_get(gid,'latitudeOfLastGridPointInDegrees')
            Ni=grib_get(gid,'Ni')
            Nj=grib_get(gid,'Nj')


            values=grib_get_values(gid)
            elat=grib_get_array(gid,'latitudes')
            elon=grib_get_array(gid,'longitudes')
    

            gridType = grib_get(gid,'gridType')

            lat0 = grib_get(gid,'latitudeOfSouthernPoleInDegrees')
            lon0 = grib_get(gid,'longitudeOfSouthernPoleInDegrees')


            elon = elon.reshape(Nj,Ni)
            elat = elat.reshape(Nj,Ni)
            q = values.reshape(Nj,Ni)
    
            prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
            [[a0,a1],[a2,a3]] = prj([minlon, minlat], [maxlon, maxlat])

            orig = geometry.SwathDefinition(lons=elon,lats=elat) # original points
    
    
            area_id = 'HNMS'
            description = 'HNMS COSMO'
            proj_id = 'eqc'
            projection = '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
            x_size = Ni
            y_size = Nj
            area_extent = (a0, a1, a2, a3)
            target_def = utils.get_area_def(area_id, description, proj_id, projection,
                                       x_size, y_size, area_extent)

            lons,lats = geometry.AreaDefinition.get_lonlats(target_def)
    
            data = bilinear.resample_bilinear(q, orig, target_def,
                                      radius=50e3, neighbours=32,
                                      nprocs=1, fill_value=0,
                                      reduce_data=False, segments=None,
                                      epsilon=0)
    
            timestamp =  datetime.datetime.strptime(str(date),'%Y%m%d')+datetime.timedelta(hours=dataTime/100) 


            if name == 'msl' : 
                     pt.append(data)
                     tt.append(timestamp+datetime.timedelta(hours=int(stepRange)))
            elif name == '10u':
                     ut.append(data)
            elif name == '10v':
                     vt.append(data)


    # END OF FOR

    except Exception as e:
        print e
        print 'ERROR in meteo file {}'.format(date)

    f.close()

met = xr.Dataset({'msl': (['time', 'latitude', 'longitude'],  np.array(pt)), 
                          'u10': (['time', 'latitude', 'longitude'], np.array(ut)),   
                          'v10': (['time', 'latitude', 'longitude'], np.array(vt)),   
                          'lons': (['x', 'y'], lons),   
                          'lats': (['x', 'y'], lats)},   
                          coords={'longitude': ('longitude', lons[0,:]),   
                                  'latitude': ('latitude', lats[:,0]),   
                                  'time': tt })   

In [ ]:
met

In [ ]:
met['msl'] = met.msl.where(met.msl>0)

In [ ]:
met['u10'] = met.u10.where(met.msl>0)
met['v10'] = met.v10.where(met.msl>0)

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

from mpl_toolkits.axes_grid1 import make_axes_locatable


proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(lons,lats,met.msl[0,], transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

In [ ]:
met.dims.keys()[:3], met.variables.keys()[:3]

In [ ]:
dp = hv.Dataset(met,kdims=met.dims.keys()[:3],vdims=met.variables.keys()[:3])

In [ ]:
%%opts Image [width=650 height=450] (cmap='jet')
dp.to(hv.Image, ['longitude','latitude'], 'msl').hist()

In [ ]:
gp = gv.Dataset(met,kdims=met.dims.keys()[:3],vdims=met.variables.keys()[:3])

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above"  tools=['hover']] (cmap='viridis')
gp.to(gv.Image, ['longitude','latitude'], 'msl', dynamic=True) * gf.coastline() * gf.borders()